# import

In [1]:
import pandas as pd
import numpy as np
import os
from keras import models
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import yaml

2025-03-06 16:07:14.825810: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-06 16:07:14.825834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-06 16:07:14.826582: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-06 16:07:14.830834: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-06 16:07:15.400669: W tensorflow/compiler/tf2

# functions

In [2]:
def load_config(config_path="config.yaml"):
    """
    Loads configuration from a YAML file.

    Parameters:
    - config_path (str): Path to the YAML configuration file.

    Returns:
    - dict: Dictionary with configuration parameters.
    """
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
    return config

# params

In [3]:
proj_path="/media/jczars/4C22F02A22F01B22/Pollen-Classification-CBAM/"

In [4]:
params = load_config("/media/jczars/4C22F02A22F01B22/Pollen-Classification-CBAM/phase3/config_test_views.yaml")
print(params)

{'path_model': 'results/phase1/AT_densenet+cbam_exp/0_DenseNet201/', 'path_full_labels': '/media/jczars/4C22F02A22F01B22/$WinREAgent/Pollen_classification_view/BD/CPD1_Cr_Rs_400/Test/k1', 'path_test': 'BD/CPD1_TEST_VIEW/', 'save_dir': 'results/phase3/', 'save': 1, 'verbose': 1, 'size': 224}


# categories

In [5]:

categories = sorted(os.listdir(params['path_full_labels']))
for idx, i in enumerate(categories):
    print(idx, i)

0 asphodelus
1 calicotome
2 castanea
3 ceratonia
4 ebenus
5 erica
6 eucalyptus
7 ferula
8 myrtus
9 olea
10 origanum
11 oxalis
12 pinus
13 pistacia
14 salvia
15 satureja
16 sinapis
17 thymbra
18 urginea
19 vitis


In [6]:
path=params['path_test']
path

'BD/CPD1_TEST_VIEW/'

In [7]:
name_test = os.path.basename(os.path.normpath(path))  # Obtém o último diretório do caminho
name_test

'CPD1_TEST_VIEW'

In [8]:
path_save=f"{params['save_dir']}{name_test}/"
print(f"path_dst: {path_save}")

path_dst: results/phase3/CPD1_TEST_VIEW/


In [9]:
os.makedirs(path_save, exist_ok=True) 

In [10]:
path=params['path_model']
name_model = os.path.basename(os.path.normpath(path))  # Obtém o último diretório do modelo
path_save=f"{path_save}{name_model}/"
os.makedirs(path_save, exist_ok=True) 
print(f"path_dst: {path_save}")

path_dst: results/phase3/CPD1_TEST_VIEW/0_DenseNet201/


In [11]:
k=1

# Load model

In [12]:
path_model = params['path_model']
nm_model= path_model.split("/")[-2]
path = f"{proj_path}{path_model}{nm_model}_bestLoss_{k}.keras"
print(f"\n[INFO] path_model: {path}\n")


[INFO] path_model: /media/jczars/4C22F02A22F01B22/Pollen-Classification-CBAM/results/phase1/AT_densenet+cbam_exp/0_DenseNet201/0_DenseNet201_bestLoss_1.keras



In [13]:
model=None
model = models.load_model(path)
model.summary()  # Print model summary

2025-03-06 16:07:16.976681: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-06 16:07:17.008675: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-06 16:07:17.008821: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 230, 230, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1/conv[0][0]']      

# Functions

In [14]:
def load_data_test_vw(test_dir, input_size, batch_size=32):
    idg = ImageDataGenerator(rescale=1.0 / 255)
    test_generator = idg.flow_from_directory(
        test_dir,
        target_size=input_size,
        class_mode="categorical",
        shuffle=False,
        batch_size=batch_size
    )
    return test_generator

def generate_classification_report_vw(model, cat_model, test_generator, output_csv="relatorio.csv"):
    """
    Gera um relatório de classificação e salva como CSV.
    """
    y_true = test_generator.classes  
    test_class_names = list(test_generator.class_indices.keys())  
    label_mapping = {test_class_names[i]: cat_model.index(test_class_names[i]) for i in range(len(test_class_names))}
    y_true_mapped = np.array([label_mapping[test_class_names[i]] for i in y_true])
    predictions = model.predict(test_generator, verbose=1)
    y_pred = np.argmax(predictions, axis=1)
    present_labels = sorted(set(y_true_mapped))  
    report = classification_report(y_true_mapped, y_pred, labels=present_labels, 
                                   target_names=[cat_model[i] for i in present_labels], 
                                   output_dict=True, zero_division=0)
    df_report = pd.DataFrame(report).transpose()
    df_report.to_csv(output_csv)
    print(f"✅ Relatório salvo em: {output_csv}")

def generate_confusion_matrix_vw(model, cat_model, test_generator, output_image="matriz_confusao.png", normalize=False):
    """
    Gera e exibe a matriz de confusão com destaque para classes ausentes e anotação de valores fora da diagonal principal.
    """
    y_true = test_generator.classes  
    test_class_names = list(test_generator.class_indices.keys())  
    label_mapping = {test_class_names[i]: cat_model.index(test_class_names[i]) for i in range(len(test_class_names))}
    y_true_mapped = np.array([label_mapping[test_class_names[i]] for i in y_true])
    predictions = model.predict(test_generator, verbose=1)
    y_pred = np.argmax(predictions, axis=1)
    present_labels = sorted(set(y_true_mapped))  
    absent_labels = sorted(set(range(len(cat_model))) - set(present_labels))  
    cm = confusion_matrix(y_true_mapped, y_pred, labels=present_labels)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    full_cm = np.zeros((len(cat_model), len(cat_model)))
    for i, real in enumerate(present_labels):
        for j, pred in enumerate(present_labels):
            full_cm[real, pred] = cm[i, j]
    
    fig, ax = plt.subplots(figsize=(12, 10))
    cmap = sns.color_palette("Blues", as_cmap=True)  
    sns.heatmap(full_cm, annot=True, fmt=".2f" if normalize else "g", cmap=cmap, 
                xticklabels=cat_model, yticklabels=cat_model, cbar=True, linewidths=0.5, ax=ax)
    
    for i in absent_labels:
        ax.get_yticklabels()[i].set_backgroundcolor("yellow")  
        ax.get_xticklabels()[i].set_backgroundcolor("yellow")  
    
    for i in range(len(full_cm)):
        for j in range(len(full_cm)):
            value = full_cm[i, j]
            if i != j and value > 0:  
                ax.add_patch(plt.Rectangle((j, i), 1, 1, fill=True, color='lightcoral', alpha=0.5))
                ax.text(j + 0.5, i + 0.5, f'{value:.2f}' if normalize else f'{int(value)}', 
                        ha='center', va='center', color='black', fontsize=10)
    
    plt.xlabel("Predito")
    plt.ylabel("Real")
    plt.title("Matriz de Confusão" + (" (Normalizada)" if normalize else "") + "\n(Fundo Amarelo = Classes Ausentes no Teste)")
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig(output_image)
    print(f"✅ Matriz de confusão salva em: {output_image}")
    plt.show()

# Equatorial view

In [15]:
view="EQUATORIAL"
path_test=f"{params['path_test']}k{k}/"
view_dir = f"{proj_path}{path_test}{view}/" 
print(f"view_dir: {view_dir}")

view_dir: /media/jczars/4C22F02A22F01B22/Pollen-Classification-CBAM/BD/CPD1_TEST_VIEW/k1/EQUATORIAL/


In [16]:
input_size = (224, 224)  

test_generator = load_data_test_vw(view_dir, input_size)

Found 210 images belonging to 16 classes.


In [17]:
y_true = test_generator.classes  
y_true

array([ 0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,
        3,  3,  4,  5,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,
        9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15,
       15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
       15, 15, 15, 15, 15, 15], dtype=int32)

In [18]:
filenames = test_generator.filenames
filenames

['asphodelus/Asp.A.ramX400wF(1984)(0)A-10.png',
 'asphodelus/Asp.A.ramX400wF(1987)(0)D-1.png',
 'castanea/Fag.C.satX400wF(781)(0)A-2.png',
 'castanea/Fag.C.satX400wF(781)(0)A-22.png',
 'castanea/Fag.C.satX400wF(781)(0)A-23.png',
 'castanea/Fag.C.satX400wF(781)(0)A-31.png',
 'castanea/Fag.C.satX400wF(781)(0)A-33.png',
 'castanea/Fag.C.satX400wF(781)(0)A-7.png',
 'castanea/Fag.C.satX400wF(781)(0)A-8.png',
 'castanea/Fag.C.satX400wF(782)(0)B-5.png',
 'castanea/Fag.C.satX400wF(782)(0)B-7.png',
 'castanea/Fag.C.satX400wF(784)(0)D-11.png',
 'castanea/Fag.C.satX400wF(786)(0)F-10.png',
 'castanea/Fag.C.satX400wF(786)(0)F-2.png',
 'castanea/Fag.C.satX400wF(792)(781)G-18.png',
 'castanea/Fag.C.satX400wF(792)(781)G-20.png',
 'castanea/Fag.C.satX400wF(792)(781)G-26.png',
 'castanea/Fag.C.satX400wF(792)(781)G-31.png',
 'castanea/Fag.C.satX400wF(792)(781)G-41.png',
 'castanea/Fag.C.satX400wF(792)(781)G-49.png',
 'castanea/Fag.C.satX400wF(793)(781)H-45.png',
 'castanea/Fag.C.satX400wF(793)(781)H-52.p

In [42]:
df=None

In [43]:
df = pd.DataFrame(filenames, columns=['file'])
confidences = [] 

In [44]:
test_class_names = list(test_generator.class_indices.keys())  
test_class_names

['asphodelus',
 'castanea',
 'ceratonia',
 'ebenus',
 'erica',
 'eucalyptus',
 'ferula',
 'olea',
 'origanum',
 'pistacia',
 'salvia',
 'satureja',
 'sinapis',
 'thymbra',
 'urginea',
 'vitis']

In [45]:
categories = sorted(os.listdir(params['path_full_labels']))
for idx, i in enumerate(categories):
    print(idx, i)

0 asphodelus
1 calicotome
2 castanea
3 ceratonia
4 ebenus
5 erica
6 eucalyptus
7 ferula
8 myrtus
9 olea
10 origanum
11 oxalis
12 pinus
13 pistacia
14 salvia
15 satureja
16 sinapis
17 thymbra
18 urginea
19 vitis


In [46]:
label_mapping = {test_class_names[i]: categories.index(test_class_names[i]) for i in range(len(test_class_names))}
label_mapping

{'asphodelus': 0,
 'castanea': 2,
 'ceratonia': 3,
 'ebenus': 4,
 'erica': 5,
 'eucalyptus': 6,
 'ferula': 7,
 'olea': 9,
 'origanum': 10,
 'pistacia': 13,
 'salvia': 14,
 'satureja': 15,
 'sinapis': 16,
 'thymbra': 17,
 'urginea': 18,
 'vitis': 19}

In [47]:
y_true_mapped = np.array([label_mapping[test_class_names[i]] for i in y_true])
y_true_mapped

array([ 0,  0,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,
        4,  4,  5,  6,  7,  7,  7,  7,  7,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 13, 13,
       13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19,
       19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
       19, 19, 19, 19, 19, 19])

In [48]:
len(y_true_mapped)

210

In [49]:
y_preds = model.predict(test_generator, verbose=1)


7/7 [==============================] - 1s 91ms/step


In [50]:
len(y_preds)

210

In [51]:
for prediction in y_preds:
    confidence = np.max(prediction)
    confidences.append(confidence)

In [52]:
len(confidences)

210

In [53]:
y_pred = np.argmax(y_preds, axis=1)
present_labels = sorted(set(y_true_mapped))  
present_labels

[0, 2, 3, 4, 5, 6, 7, 9, 10, 13, 14, 15, 16, 17, 18, 19]

In [54]:
print(f'Size y_true: {len(y_true)}')
print(f'Size y_pred: {len(y_pred)}')

Size y_true: 210
Size y_pred: 210


In [55]:
df['y_true'] = y_true_mapped

In [56]:
df['y_pred'] = y_pred

In [57]:
df['confidence'] = confidences
df['true_label'] = [categories[i] for i in y_true_mapped]
df['predicted_label'] = [categories[i] for i in y_pred]

In [58]:
df['status'] = df.apply(lambda row: 'Correct' if row['y_true'] == row['y_pred'] else 'Incorrect', axis=1)

# Separate the DataFrame into correct and incorrect predictions
df_correct = df[df['status'] == 'Correct']    
df_incorrect = df[df['status'] == 'Incorrect']

In [ ]:
df_correct

,file,y_true,y_pred,confidence,true_label,predicted_label,status
0,asphodelus/Asp.A.ramX400wF(1984)(0)A-10.png,0,0,0.999997,asphodelus,asphodelus,Correct
1,asphodelus/Asp.A.ramX400wF(1987)(0)D-1.png,0,0,0.996568,asphodelus,asphodelus,Correct
2,castanea/Fag.C.satX400wF(781)(0)A-2.png,2,2,0.999803,castanea,castanea,Correct
3,castanea/Fag.C.satX400wF(781)(0)A-22.png,2,2,0.999874,castanea,castanea,Correct
4,castanea/Fag.C.satX400wF(781)(0)A-23.png,2,2,0.999926,castanea,castanea,Correct
...,...,...,...,...,...,...,...
205,vitis/Vit.V.viniX400wF(4984)(0)G-37.png,19,19,0.999554,vitis,vitis,Correct
206,vitis/Vit.V.viniX400wF(4984)(0)G-7.png,19,19,0.999884,vitis,vitis,Correct
207,vitis/Vit.V.viniX400wF(4984)(0)G-9.png,19,19,0.999678,vitis,vitis,Correct
208,vitis/Vit.V.viniX400wF(4985)(0)H-10.png,19,19,0.999877,vitis,vitis,Correct


: 